In [43]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
import json
import sys
import re
import logging
import os
from pathlib import Path
from functools import partial
from itertools import chain
from uuid import uuid4
from threading import Thread

import blpapi
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr, VegaExpr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots

import bhutils
from bhutils import bhutilspy as bh
# from api.ezutils import bh, mkt_defs, ccy_cals, mx_cals, mtg_dts, gens

from gioutils.utils import parse_offset, today, get_next_n, get_bday
from gioutils.gui.base import View, ViewStore, WidgetView, Tabs
from gioutils.blpw import BlpQuery
from gioutils.utils import RepeatedTimer, RepeatedTimerThread
from gioutils import ezutils as ez
from gioutils.ezutils import bh, bhs, SQL, ZCILSHelper, get_price_info, get_id, LiveInflLocal, COBInflLocal
from gioutils.gui.base import ViewStore
from gioutils.blpw import BlpParser

from api.data.base import data_path, radar_path
from api.gui.params import bbg_params, spot_params, fwd_params
from api.gui.theme import bg_color, grid_style
from api.gui.pandas import DFOutput, negative_red, custom_styler_row_line, DataGridGio
from api.collectors import CollectorStoreMixin, SpotForward, Futures, Market, Mtgs, FwdMatrix, VolMatrix, ECOPred, ZCILMatrix, ECO
from api.gui.editable_grid import EditableGrid
from api.gui.price_grids import PricingGrid
from api.gui.grids import SpotGrid, FutureGrid, MtgGrid, MatGrid, GroupMktGrid # GroupMktTalbe, KeyChart, MtgTable, MatTable, CustomGovtBondFutTable
from api.gui.global_config import GlobalConfig
from api.gui.monitors import MktMonitor, VolMonitor, ECOMonitor
from api.app import DataCollector
from api.gui.renderers import RendererStoreMixin, RendererComponentStoreMixin


pd.options.plotting.backend = "plotly"

In [45]:
bq = BlpQuery(timeout=50000, parser=BlpParser(raise_security_errors=False)).start()

In [46]:
ccys = [
    'EUR', 
    'USD', 
    'GBP',
    # 'JPY'
]

In [47]:
pricer_grid = PricingGrid()
mm = MktMonitor()
vm = VolMonitor()
em = ECOMonitor()

In [48]:
eco_fci = ECOPred(
    X_names= [
            'GT2 Govt',
            'GT10 Govt',
            'ES1 Index',
            'CDX HY CDSI GEN 5Y SPRD Curncy',
            'CDX IG CDSI GEN 5Y SPRD Curncy',
            'CL1 Comdty',
        ],
    y_name = 'NFCIINDX Index',
    name='nfci',
    display_name='NFCI',
    
)

In [49]:
sf = SpotForward(spot_params=spot_params, fwd_params=fwd_params, name='sf')
pg = pricer_grid = PricingGrid(name='pg')
fut = Futures(name='fut')
mkt = Market(name='mkt')
mtgs = Mtgs(name='mtgs')
mat = FwdMatrix(name='mat'),
vol_mat = VolMatrix(name='vol_mat')
zcil_mat = ZCILMatrix(name='zcil_mat')
eco = ECO(name='eco')

In [50]:
live_infl = LiveInflLocal()
cob_infl = COBInflLocal()

In [51]:
dc = DataCollector(
    live_infl=live_infl,
    cob_infl=cob_infl,
    ccys=None,
    bq=bq,
    
)

In [52]:
self = dc

In [ ]:
threads = []
res = {}
for i in range(2):
    thread = Thread(target=dl, args=(bq, i, res), daemon=True)
    thread.start()
    threads.append(thread)

In [59]:
snaps = self.get_snaps()
bq = self.bq.start()


threads = []
print('start for')
for name, collector in self.collectors.items():
    # kwargs = collector.get_collector_kwargs()
    thread = Thread(
        target=collector.get_data, 
        kwargs=dict(
            snaps=snaps,
            cob_date=self.cob_curves.date,
            bq=bq
        ), 
        daemon=True
    )
    thread.start()
    threads.append(thread)
print('end for')
[thread.join() for thread in threads]
print('end join')

start for
end for


end join


[None, None, None, None, None, None, None, None, None]

In [10]:
gc = GlobalConfig(dc=dc, timer=RepeatedTimerThread)

In [11]:
tabs = Tabs(
    {
        # 'radar': v.Container(
        #     fluid=True,
        #     children=[
        #         v.Row(children=[v.Col(children=[gc.view])]),
        #         v.Row(children=[v.Col(children=[mm.view])])
        #     ],
        # ),
        'radar':v.Container(
            fluid=True,
            children=[mm.view]
        ),
        'vol':v.Container(
            fluid=True,
            children=[vm.view]
        ),
        'eco':  v.Container(
            fluid=True,
            children=[em.view]
        ),
        'pricer': v.Container(
            fluid=True,
            children=[w.VBox([pricer_grid.box, pricer_grid.price_grid.view])],
        ),
        'monitor': pricer_grid.expr_dg.view,
        # 'a': w.HTML('test'),
        # 'b': w.HTML('test'),
        # 'c': w.HTML('test'),
    },
    force_grid_render=False,
    # tabs_kwargs={'class_': "mb-2"}
)

In [12]:
v.Container(
    children=[
        # gc.view,
        v.Row(
            children=[
                v.Col(children=[gc.view], offset=6, class_='my-0 py-0'),
            ],
        ),
        v.Row(
            children=[
                v.Col(children=[tabs.view], class_='ma-0 pa-0'),
            ],
        ),
    ],
    fluid=True,
    class_='ma-0 pa-0'
)

Container(children=[Row(children=[Col(children=[Row(children=[Col(children=[TextField(clearable=True, dense=Tr…